In [1]:
import tensorflow as tf
import numpy as np

In [2]:
a = tf.constant([[1,2,3],[4,4,5]])
a
# print("Shape: ",a.shape)

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 4, 5]], dtype=int32)>

In [3]:
b = tf.constant(42)
b
# print(b)
# print("Shape: ",b.shape)

<tf.Tensor: shape=(), dtype=int32, numpy=42>

In [4]:
a[...,2,tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=int32, numpy=
array([[3],
       [5]], dtype=int32)>

In [5]:
print(a*b)

tf.Tensor(
[[ 42  84 126]
 [168 168 210]], shape=(2, 3), dtype=int32)


In [6]:
print(a+b)

tf.Tensor(
[[43 44 45]
 [46 46 47]], shape=(2, 3), dtype=int32)


In [7]:
a = np.array([1,2,3])

In [8]:
t = tf.constant(a)
t.numpy()
tf.square(a)

<tf.Tensor: shape=(3,), dtype=int64, numpy=array([1, 4, 9])>

In [9]:
b = tf.constant(2.,dtype='float64')+tf.constant(3.,dtype='float64')

In [10]:
x = tf.Variable([[1,2],[3,4]])
x.assign_add([[5,6],[4,1]])

<tf.Variable 'UnreadVariable' shape=(2, 2) dtype=int32, numpy=
array([[6, 8],
       [7, 5]], dtype=int32)>

In [11]:
def huber_fn(y_true, y_pred):
    error = (y_true - y_pred)
    is_small_error = tf.abs(error) < 1
    squared_error = tf.square(error)/2
    linear_error = tf.abs(error)-0.5

    return tf.where(is_small_error,squared_error,linear_error)

In [12]:
huber_fn(0.8,0.7)

<tf.Tensor: shape=(), dtype=float32, numpy=0.0050000004>

In [13]:
import keras

In [14]:
class ResidualBlock(keras.layers.Layer):
    def __init__(self,n_layers,n_units,**kwargs):
        self.hidden = [ keras.layers.Dense(n_units, activation='elu',
                                           kernel_initializer='he_normal')
                        for _ in range(n_layers)]
    
    def call(self,inputs):
        z = inputs
        for layer in self.hidden:
            z = layer(z)
        return z + inputs

In [15]:
class ResidualRegressor( keras.Model):
    def __init__(self,output_dims,**kwargs):
        super.__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(30,activation='elu',kernel_initializer='he_normal')
        self.hidden2 = ResidualBlock(2,30)
        self.hidden3 = ResidualBlock(2,30)
        self.output = keras.layers.Dense(output_dims)
    
    def call(self,inputs):
        x = self.hidden1(inputs)
        for i in range(1+3):
            x = self.hidden2(x)
        x = self.hidden3(x)
        return self.output(x)


In [16]:
import tensorflow as tf
def f(w1,w2):
    return 3*w1**2 + 2*w1*w2

w1, w2 = tf.Variable(5.0), tf.Variable(3.0)
with tf.GradientTape(persistent=True) as tape:
    z = f(w1,w2)

gradients = tape.gradient(z,[w1,w2])

gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

In [19]:
dz_dw1 = tape.gradient(z,w1)
dz_dw1

dz_dw2 = tape.gradient(z, w2)

NameError: name 'tape' is not defined

In [20]:
l2_reg = keras.regularizers.l2(0.05)
model = keras.Sequential([
    keras.layers.Dense(30, activation='elu',kernel_initializer='he_normal',kernel_regularizer=l2_reg),
    keras.layers.Dense(1,kernel_regularizer=l2_reg)
])

In [21]:
def random_batch(X,y,batch_size=32):
    idx = np.random.randint(len(X),size=batch_size)
    return X[idx], y[idx]


In [22]:
def print_statusbar(iteration,total,loss,metrics=None):
    metrics = " - ".join(["{}:{:.4f}".format(m.name,m.result) for m in [loss]+(metrics or [])])
    end = " " if iteration < total else "\n"
    print("\r{}/{}".format(iteration,total) + metrics, end=end)

In [23]:
def cube(x):
    return x**3

In [26]:
tf_cube = tf.function(cube)
tf_cube

In [28]:
tf_cube(5.12)

<tf.Tensor: shape=(), dtype=float32, numpy=134.21773>

In [30]:
tf_cube.python_function(2)

8

In [1]:
import tensorflow as tf

In [14]:
tensor_data = tf.constant([1,2,3,4,5])
dataset = tf.data.Dataset.from_tensor_slices(tensor_data)
dataset = dataset.repeat(3).batch(2)
dataset = dataset.apply(tf.data.experimental.unbatch())
dataset = dataset.filter( lambda x : x<5)
for item in dataset:
    print(item.numpy())

1
2
3
4
1
2
3
4
1
2
3
4


2024-06-24 11:00:43.598335: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [8]:
file = ['text.txt']
dataset = tf.data.TextLineDataset(file)

dataset = dataset.batch(5)

for line in dataset:
    print(line)

tf.Tensor([b'Hi my name is Yash'], shape=(1,), dtype=string)


2024-06-24 10:45:42.649972: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [1]:
#interleaving lines
import tensorflow as tf

In [2]:
# Sample data
data1 = ["file1_line1", "file1_line2", "file1_line3"]
data2 = ["file2_line1", "file2_line2", "file2_line3"]
data3 = ["file3_line1", "file3_line2", "file3_line3"]

In [3]:
def create_dataset(data):
    return tf.data.Dataset.from_tensor_slices(data)

In [4]:
dataset1 = create_dataset(data1)
dataset2 = create_dataset(data2)
dataset3 = create_dataset(data3)

In [5]:
#list of datasets
datasets = [ dataset1, dataset2, dataset3]

In [6]:
#interleaving dataset
interleaved_dataset = tf.data.Dataset.from_tensor_slices(datasets).interleave(
    lambda x: x,
    cycle_length = 3,
    block_length = 1
)

In [8]:
for item in interleaved_dataset:
    print(item.numpy().decode('utf-8'))

2024-06-25 10:23:50.302617: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: NOT_FOUND: file1_line1; No such file or directory


NotFoundError: {{function_node __wrapped__IteratorGetNext_output_types_1_device_/job:localhost/replica:0/task:0/device:CPU:0}} file1_line1; No such file or directory [Op:IteratorGetNext] name: 

In [12]:
import tensorflow as tf

# Sample data
data1 = ["file1_line1", "file1_line2", "file1_line3"]
data2 = ["file2_line1", "file2_line2", "file2_line3"]
data3 = ["file3_line1", "file3_line2", "file3_line3"]

# Function to create a dataset from a list of strings
def create_dataset(data):
    return tf.data.Dataset.from_tensor_slices(data)

# Create individual datasets
dataset1 = create_dataset(data1)
dataset2 = create_dataset(data2)
dataset3 = create_dataset(data3)

# List of datasets
datasets = [dataset1, dataset2, dataset3]

# Interleave the datasets
interleaved_dataset = tf.data.Dataset.from_tensor_slices(datasets).interleave(
    lambda x: x,
    cycle_length=4,  # Number of datasets to interleave
    block_length=4,
    num_parallel_calls = 3   # Number of consecutive elements to produce from each input element before cycling to the next input element
)

# Print the elements of the interleaved dataset
for element in interleaved_dataset:
    print(element.numpy().decode('utf-8'))


file1_line1
file1_line2
file1_line3
file2_line1
file2_line2
file2_line3
file3_line1
file3_line2
file3_line3


2024-06-25 10:27:54.711664: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [ ]:
def csv_reader_dataset(filepaths, repeat=1, n_readers = 5,
                       n_read_threads = None, shuffle_buffer_size = 10000,
                       n_parse_threads = 5, batch_size = 32 ):
    dataset = tf.data.Dataset.list_files(filepaths)
    dataset = dataset.interleave(
        lambda filepath : tf.data.TextLineDataset(filepath).skip(1),
        cycle_length=n_readers, num_parallel_calls=n_read_threads)
    dataset = dataset.map(proprocess, num_parallel_calls=n_parse_threads)
    dataset = dataset.shuffle(shuffle_buffer_size).repeat(repeat)
    return dataset.batch(batch_size).prefetch(1)

In [28]:
options = tf.io.TFRecordOptions(compression_type='GZIP')
with tf.io.TFRecordWriter('mydata_compressed.tfrecord',options) as f:
	f.write(b"hi my names is Yash")
	f.write(b"I am studying Machine learning and Artificial intelligence")

In [30]:
filepath = [ 'mydata_compressed.tfrecord']
dataset = tf.data.TFRecordDataset(filepath,compression_type="GZIP")
for item in dataset: 
	print(item)

tf.Tensor(b'hi my names is Yash', shape=(), dtype=string)
tf.Tensor(b'I am studying Machine learning and Artificial intelligence', shape=(), dtype=string)


2024-06-25 11:23:48.794374: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [1]:
import tensorflow as tf

In [2]:
vocab = [ '<H OCEAN', 'INLAND', 'NEAR OCEAN', 'NEAR BAY', 'ISLAND']
indices = tf.range(len(vocab),dtype=tf.int64)
table_init = tf.lookup.KeyValueTensorInitializer(vocab, indices)
num_oov = 2
table = tf.lookup.StaticVocabularyTable(table_init,num_oov_buckets=num_oov)

In [6]:
category = tf.constant([ '<H OCEAN', 'DESERT', 'NEAR OCEAN', 'NEAR BAY', 'ISLAND'])
cat_indices = table.lookup(category)
print(cat_indices)

tf.Tensor([0 5 2 3 4], shape=(5,), dtype=int64)


In [7]:
cat_one_hot = tf.one_hot(cat_indices, depth=len(vocab)+num_oov)

In [8]:
cat_one_hot

<tf.Tensor: shape=(5, 7), dtype=float32, numpy=
array([[1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.]], dtype=float32)>